In [3]:
# gen data

import os
import re
import json
import copy
import time
import logging
import random
import pickle

from utils.chat2DeepSeek import Chat2DeepSeek
from utils.gen_prompt import Prompt
from utils.util import *

In [8]:
API_KEY = 'sk-7015d98e24c9431f9fb7fb2a4454186e'
event_sen = {}
event_sub_list = ['SUE']#

# event_sen = load_json('./data_5_ATTACK_6_15_18_53.json')
data4chat = load_pickle('./exp_data/event_batch_withexam10.pkl')
event_dict = load_json('./meta_data/event_dict_full.json') # defin from doc
trigger_dict = load_json('./exp_data/trigger_sout_top10.json')
arguments_dict = load_json('./exp_data/argument_sout_tophalf.json')

fp = './meta_data/arg_roles/'

arg_role_definition = {}
for fn in os.listdir(fp):
    if not fn.endswith('.txt'):
        continue
    with open(fp + fn, 'r', encoding = 'utf-8') as f:
        s = f.read()
    j = json.loads(s)
    arg_role_definition.update(j)

In [10]:
n_trigger = 100
n_argument = 20
sentence_per_event = 50
complex_score = [5, 10] # not implemented
max_argument = 2
max_event = 5
max_refine = 3

weight_dict = {0:[1],
                1:[1],
                2:[0.5, 0.5],
                3:[0.2, 0.5, 0.3],
                4:[0.1, 0.2, 0.3, 0.4],
                5:[0.1, 0.1, 0.3, 0.3, 0.2]
                  }

In [12]:
def get_n_event(max_event, weight_dict):
    if max_event == 0:
        return 0
    return random.choices([ i for i in range(1, max_event + 1)], weights=weight_dict[max_event])

def get_local_time():
    return str(time.localtime().tm_mon) + '_' + str(time.localtime().tm_mday) + '_' + str(time.localtime().tm_hour) + '_' + str(time.localtime().tm_min)
    

In [91]:
def narrator_prompt(event_record, event_dict):
    task = '''
Task Definition 
    You are an writer, looking to write sentences that contain specific events and event triggers. An event is a specific occurrence involving participants. An event is something that happens. An event can frequently be described as a change of state. Event trigger is the word that most clearly expresses its occurrence. Event triggers are often only a few words in length. 
    '''
    ontology = 'Related Event Ontology\n\t'
    query = '\nQuery \n\tGenerate a new sentence using '
    for e_id, r in event_record.items():
        event = r['event']
        trigger = r['trigger']
        ontology += f'\n\tA "{event}" event means: {event_dict[event][0]}'
        query += f'trigger "{trigger}" for event {event}, '
    output = '\nOutPut Format: \n\toutput in json, e.g., {"sentences": "generated sentences"}'
    return task + ontology + query[:-2] + '.' + output
    
def refiner_prompt(event_record, sen, event_dict):
    task = '''
Task Definition 
    This is an event extraction task where the goal is to extract structured events from the text. A structured event contains an event trigger word and an event type. 
Full Event Ontology'''
    ontology =  '\n\tEvents of interests:\n\t'
    for e_id, r in event_record.items():
        event = r['event']
        trigger = r['trigger']
        ontology += f'\n\tA "{event}" event means: {event_dict[event][0]}'
    query = '''
Query
    Below is a sentence from which you need to extract the events if any. Only output a list of tuples in the form [("event_type_1", "event_trigger_word_1"), ("event_type_2", "event_trigger_word_2"), ...] for each event in the sentence. 
    sentence: 
    '''
    query += f'"{sen}".'
    return task + ontology + query


In [93]:
gen_prompt = Prompt(
    event_dict = event_dict, 
    trigger_dict = trigger_dict, 
    arguments_dict = arguments_dict, 
    n_event = get_n_event(max_event, weight_dict)[0], 
    max_argument = max_argument,
    complex_score = complex_score,
    use_chain = False)

In [95]:
prompt, record = gen_prompt.gen_data_prompt(main_event='ATTACK', n_event=5)
record = json.loads(record)

In [97]:
print(narrator_prompt(record, event_dict))


Task Definition 
    You are an writer, looking to write sentences that contain specific events and event triggers. An event is a specific occurrence involving participants. An event is something that happens. An event can frequently be described as a change of state. Event trigger is the word that most clearly expresses its occurrence. Event triggers are often only a few words in length. 
    Related Event Ontology
	
	A "ATTACK" event means: An ATTACK Event is defined as a violent physical act causing harm or damage.  ATTACK Events include any such Event not covered by the INJURE or DIE subtypes, including Events where there is no stated agent.  The ATTACK Event type includes less specific violence-related nouns such as ‘conflict’, ‘clashes’, and ‘fighting’.  ‘Gunfire’, which has the qualities of both an Event and a weapon, should always be tagged as an ATTACK Event, if only for the sake of consistency.  A ‘coup’ is a kind of ATTACK (and so is a ‘war’). 
	A "ARREST-JAIL" event means:

In [102]:
sen = "During the intense fighting in the city, rebels fired at government forces, leading to several soldiers being detained; later, their Trials were swift, and three were hanged for treason."
print(refiner_prompt(record,sen, event_dict))


Task Definition 
    This is an event extraction task where the goal is to extract structured events from the text. A structured event contains an event trigger word and an event type. 
Full Event Ontology
	Events of interests:
	
	A "ATTACK" event means: An ATTACK Event is defined as a violent physical act causing harm or damage.  ATTACK Events include any such Event not covered by the INJURE or DIE subtypes, including Events where there is no stated agent.  The ATTACK Event type includes less specific violence-related nouns such as ‘conflict’, ‘clashes’, and ‘fighting’.  ‘Gunfire’, which has the qualities of both an Event and a weapon, should always be tagged as an ATTACK Event, if only for the sake of consistency.  A ‘coup’ is a kind of ATTACK (and so is a ‘war’). 
	A "ARREST-JAIL" event means: A JAIL Event occurs whenever the movement of a PERSON is constrained by a state actor (a GPE, its ORGANIZATION subparts, or its PERSON representatives). An ARREST Event occurs whenever a stat

In [104]:
record

{'event_1': {'event': 'ATTACK',
  'trigger': 'fire',
  'argument': {'Instrument': ['weapon'],
   'Attacker': ['brigade'],
   'Place': ['colombia']}},
 'event_2': {'event': 'ARREST-JAIL',
  'trigger': 'detained',
  'argument': {'Place': ['colombia'],
   'Agent': ['American commandos'],
   'Person': ['brigade']}},
 'event_3': {'event': 'TRIAL-HEARING',
  'trigger': 'Trials',
  'argument': {'Place': ['colombia'],
   'Adjudicator': ['Appeals', 'judge'],
   'Prosecutor': ['prosecutes', 'Immigration and Checkpoints Authority'],
   'Defendant': ['group']}},
 'event_4': {'event': 'EXECUTE',
  'trigger': 'hanged',
  'argument': {'Place': ['colombia'],
   'Person': ['killer'],
   'Agent': ['they']}},
 'event_5': {'event': 'ATTACK',
  'trigger': 'fighting',
  'argument': {'Instrument': ['weapon'],
   'Attacker': ['brigade'],
   'Place': ['colombia'],
   'Target': ['agent']}}}

In [106]:
s = '''[
  ("ATTACK", "fighting"),
  ("ATTACK", "fired"),
  ("ARREST-JAIL", "detained"),
  ("TRIAL-HEARING", "Trials"),
  ("EXECUTE", "hanged")
]'''

In [112]:
pickle.loads(s)

TypeError: a bytes-like object is required, not 'str'

In [114]:
import ast

In [116]:
event_list = ast.literal_eval(s)

In [118]:
event_list

[('ATTACK', 'fighting'),
 ('ATTACK', 'fired'),
 ('ARREST-JAIL', 'detained'),
 ('TRIAL-HEARING', 'Trials'),
 ('EXECUTE', 'hanged')]